In [2]:
from serenata_toolbox.datasets import Datasets
import pandas as pd
import requests
path = "/home/arthurmts/workspace/beijo-de-moca/data/"

## Dados Abertos Câmara dos Deputados

Para este projeto iremos acessar a base de dados disponibilizada pelo Governo Federal afim de analisar os dados referentes aos gastos dos nossos deputados e senadores. Esta política de dados abertos foi iniciada em 2016 no governo de Dilma Roussef, com o Decreto 8.777, que instaurou a [Política de Dados Abertos do Poder Executivo federal](http://www.planalto.gov.br/ccivil_03/_ato2015-2018/2016/decreto/D8777.htm).  

Esses dados estão disponibilizados nos portais de Dados Abertos do **Senado Federal** e da **Câmara dos Deputados**. 
 

### Escopo do trabalho

- Neste trabalho iremos usar alguns parâmetros para focar nossos esforços em dados mais proximos da realidade, trabalhando apenas com parlamentares do estado do Nordeste.  
  
  
- Dessa forma, conseguiremos visualizar melhor e encontrar informações a partir dos gastos e dos dados disponibilizados pelo projeto [serenata_de_amor](https://serenata.ai/), que também sera amplamente utilizado.

In [4]:
deputados_Nordeste = requests.get("https://dadosabertos.camara.leg.br/api/v2/deputados?siglaUf=MA&siglaUf=CE&siglaUf=PI&siglaUf=BA&siglaUf=SE&siglaUf=AL&siglaUf=PE&siglaUf=PB&siglaUf=RN&siglaUf=&ordem=ASC&ordenarPor=nome", headers={'Accept':'application/json'}).json()

In [17]:
nomes_deputados_nordeste = [dep['nome'] for dep in deputados_Nordeste['dados']]

In [8]:
df_deputados_serenata = pd.read_csv(path+'Ano-2018.csv', sep=';')

In [46]:
df_deputados_nordeste = df_deputados_serenata[df_deputados_serenata['txNomeParlamentar'].isin(nomes_deputados_nordeste)]

In [48]:
df_deputados_nordeste['vlrLiquido'].sum()

33557838.65

In [47]:
df_deputados_nordeste[df_deputados['txNomeParlamentar'] == 'JORGE SOLLA']['vlrLiquido'].sum()

467602.69

In [49]:
deputados_Nordeste

{'dados': [{'id': 204554,
   'uri': 'https://dadosabertos.camara.leg.br/api/v2/deputados/204554',
   'nome': 'ABÍLIO SANTANA',
   'siglaPartido': 'PL',
   'uriPartido': 'https://dadosabertos.camara.leg.br/api/v2/partidos/37906',
   'siglaUf': 'BA',
   'idLegislatura': 56,
   'urlFoto': 'https://www.camara.leg.br/internet/deputado/bandep/204554.jpg',
   'email': 'dep.abiliosantana@camara.leg.br'},
  {'id': 204560,
   'uri': 'https://dadosabertos.camara.leg.br/api/v2/deputados/204560',
   'nome': 'ADOLFO VIANA',
   'siglaPartido': 'PSDB',
   'uriPartido': 'https://dadosabertos.camara.leg.br/api/v2/partidos/36835',
   'siglaUf': 'BA',
   'idLegislatura': 56,
   'urlFoto': 'https://www.camara.leg.br/internet/deputado/bandep/204560.jpg',
   'email': 'dep.adolfoviana@camara.leg.br'},
  {'id': 160508,
   'uri': 'https://dadosabertos.camara.leg.br/api/v2/deputados/160508',
   'nome': 'AFONSO FLORENCE',
   'siglaPartido': 'PT',
   'uriPartido': 'https://dadosabertos.camara.leg.br/api/v2/partido